In [1]:
#  Importing libraries and module and some setting for notebook

import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct  #Cosine Similarity
import time

In [2]:
# Reading Data - Hofer
df_hofer=pd.read_csv("products_hofer.csv")
df_hofer=df_hofer[['Hofer_Number','Product Name (Produktvariante)','brands/producer']]

# Reading Data - OpenFoodWorld
df_ofw=pd.read_csv("0 Preliminary.csv")

In [4]:
# Dropping Name and GTIN null values
df_ofw=df_ofw.dropna(subset=['Name','GTIN'])
# Dropping GTIN duplicates
df_ofw=df_ofw.drop_duplicates(subset=['GTIN'])

In [5]:
# Setting index
df_hofer['Index']='HF '+df_hofer['Hofer_Number'].astype(str)
df_ofw['Index'] = 'OFW '+df_ofw.index.astype(str)

In [6]:
# Setting new df with brand + name
df_hofer['Product']=df_hofer['brands/producer'].fillna('')+' '+df_hofer['Product Name (Produktvariante)']
df_ofw['Product']=df_ofw['Brand'].fillna('')+' '+df_ofw['Name']

In [7]:
# Delete GTIN starting with 0 
df_ofw=df_ofw[df_ofw['GTIN'].astype(str).str[0] != '0']
# Delete Products with less than 4 characters
df_ofw=df_ofw[df_ofw['Product'].str.len()>4]

In [8]:
# Creating a concat df
frames=[df_hofer,df_ofw]
df=pd.concat(frames)
df = df[["Index", "Product","GTIN"]]

In [9]:
# PRELIMINARY STAGE OF MEMORY REDUCTION

# Get length of df_ofw
length=len(df_ofw)

# Get range of df_ofw to be splited into 6
num, div = length, 3
arr=[num // div + (1 if x < num % div else 0)  for x in range (div)]
lang=[]

# array of 6 splited ranges
for i in range(len(arr)+1):
    lang.append(sum(arr[:i]))

print(length)
print(lang)
print(arr)

1373285
[0, 457762, 915524, 1373285]
[457762, 457762, 457761]


In [10]:
df_ofw_temp1=df_ofw[:457762]
temp1=[df_ofw_temp1,df_hofer]
df1=pd.concat(temp1)
len(df1)
# df1.head(-50)

458834

In [11]:
len(df_hofer)

1072

In [12]:
len(df_ofw_temp1)

457762

In [13]:
#  ngrams(here we are taking n = 3 thus 3-gram (trigrams ) as  most room types only contain two or three words
#  used for cleaning and removing some punctuation (dots, comma’s etc) i.e.((,-./)) from a string 
#  and generate and collect all n-grams of the string.  

 
def ngrams(string, n=3):

#     string = re.sub(r' \d+%.',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


# # Testing ngrams work for verification 
# print(df['Product'].iloc[13214])
# ngrams(df['Product'].iloc[13214])


In [14]:
# After having each words split (token or  lemmas (n-gram generated items) ) into a vector and
# Scikit-learn’s  Tfidfvectorizer aim to do the same thing, which is to convert a collection of raw documents to a matrix of TF-IDF features. 
# Generate the matrix of TF-IDF (Term Frequency-Inverse Document frequency)values for each

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(df1['Product'].values.astype(str))

# # View sparse CSR matrix.
# print(tf_idf_matrix[0])

In [15]:
# calculate the similarity between two vectors of TF-IDF values the Cosine Similarity is usually used.
# result matrix in a very sparse terms and Scikit-learn deals with this nicely by returning a sparse CSR matrix.

def cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [16]:
#  Run the optimized cosine similarity function. 
#  Only stores the top 10 most similar items with a similarity above 0.75

t2 = time.time()
matches = cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.73)
t3 = time.time()-t2
print("SELFTIMED:", t3)

SELFTIMED: 6943.999190568924


In [19]:
matches_df = get_matches_df(matches, data.values.astype(str)) #data
matches_df = matches_df[matches_df['similarity'] < 0.99999] # For removing all exact matches
matches_df=matches_df.sort_values(['name'], ascending=False)

# erasing repeating items
matches_df = matches_df.drop_duplicates(subset=['similarity'],keep='first')

NameError: name 'get_matches_df' is not defined